In [3]:
import pandas as pd
import numpy as np
import random
import string
from scipy.stats import truncnorm
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Diccionarios de configuración
d1 = {
    "ds": "dataset1",
    "columns": [
        {
            "name": "area",
            "type": "category",
            "values": ["TI", "FIN", "HR"]
        },
        {
            "name": "id",
            "type": "unique"
        }
    ],
    "random": False
}

d2 = {
    "ds": "dataset2",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "area",
            "type": "foreign",
            "values": "dataset1.area"
        },
        {
            "name": "subarea",
            "type": "category",
            "values": ["SA1", "SA2", "SA3", "SA4"]
        }
    ],
    "random": False
}

d3 = {
    "ds": "dataset3",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "subarea",
            "type": "foreign",
            "values": "dataset2.id"
        },
        {
            "name": "income",
            "type": "numeric",
            "values": {"min": 20000, "max": 50000}
        },
        {
            "name": "goal",
            "type": "numeric",
            "values": {"min": 10000, "max": 30000, "std": 5000, "mean": 20000}
        }
    ],
    "random": True,
    "random_rows": 1000
}

d4 = {
    "ds": "dataset4",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "subarea",
            "type": "foreign",
            "values": "dataset3.subarea"
        },
        {
            "name": "budget",
            "type": "numeric",
            "values": {"min": 5000, "max": 20000}
        }
    ],
    "random": True,
    "random_rows": 500
}

d5 = {
    "ds": "dataset5",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "subarea",
            "type": "foreign",
            "values": "dataset4.subarea"
        },
        {
            "name": "expense",
            "type": "numeric",
            "values": {"min": 1000, "max": 15000}
        }
    ],
    "random": True,
    "random_rows": 700
}


In [5]:
# Función para generar un ID único de 16 caracteres
def generate_unique_id(length=16):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

# Función para crear un DataFrame basado en un diccionario de configuración
def create_dataframe_from_dict(config_dict, reference_datasets=None):
    columns = config_dict["columns"]
    if config_dict["random"]:
        max_length = config_dict["random_rows"]
    else:
        max_length = max(len(col["values"]) for col in columns if col["type"] == "category")

    data = {}
    for col in columns:
        if col["type"] == "category":
            if isinstance(col["values"], list):
                data[col["name"]] = np.random.choice(col["values"], max_length)
            else:
                ref_ds, ref_col = col["values"].split('.')
                if reference_datasets is None or ref_ds not in reference_datasets:
                    raise ValueError(f"Reference dataset {ref_ds} not found.")
                if ref_col not in reference_datasets[ref_ds].columns:
                    valid_columns = ', '.join(reference_datasets[ref_ds].columns)
                    raise ValueError(f"Column {ref_col} not found in {ref_ds}. Valid columns are: {valid_columns}")
                data[col["name"]] = np.random.choice(reference_datasets[ref_ds][ref_col], max_length)
        elif col["type"] == "unique":
            data[col["name"]] = [generate_unique_id() for _ in range(max_length)]
        elif col["type"] == "numeric":
            min_val = col["values"]["min"]
            max_val = col["values"]["max"]
            if "std" in col["values"] and "mean" in col["values"]:
                std = col["values"]["std"]
                mean = col["values"]["mean"]
                lower, upper = (min_val - mean) / std, (max_val - mean) / std
                data[col["name"]] = truncnorm.rvs(lower, upper, loc=mean, scale=std, size=max_length)
            else:
                data[col["name"]] = np.random.uniform(min_val, max_val, max_length)
    
    return pd.DataFrame(data)


In [6]:
# Función para construir dataframes desde una lista de configuraciones
def build_dataframes(conf_list):
    reference_datasets = {}
    dataframe_list = []

    for conf in conf_list:
        df = create_dataframe_from_dict(conf, reference_datasets)
        dataframe_list.append(df)
        reference_datasets[conf["ds"]] = df
    
    return dataframe_list

# Implementar la lógica para generar datos truncados
def generate_truncated_normal_data(mean, std, min_val, max_val, size):
    a, b = (min_val - mean) / std, (max_val - mean) / std
    return truncnorm.rvs(a, b, loc=mean, scale=std, size=size)



In [7]:
# Lista de configuraciones
conf_list = [d5, d2, d4, d1, d3]

# Construir los dataframes
dataframe_list = build_dataframes(conf_list)

In [8]:
dataframe_list[4]

,id,income,goal
0,FjF4tV5mMIuAb93W,35397.782813,24023.421138
1,xScV9NxDNwpziQJw,30593.064140,19597.716974
2,ENHsrWolCP0E1nPd,33996.189111,24455.033697
3,p38enVIKvSzWKtqZ,20339.502217,18907.333680
4,OH4NcPv1HSlFh5Fl,43581.682470,16803.815489
...,...,...,...
995,cIvphxcQCOCZtFNF,42713.856219,15782.382819
996,nVuTPiqg7Ia1kOc5,32536.761716,11719.646741
997,W4A00t8T1jQ4uGuk,29345.607926,20750.241721
998,bqi3qXOeTosoGvL9,31164.599089,18024.094881


In [33]:
# Requisito 5: Analizar combinaciones categóricas y extender el dataset
# Suponemos que dataframe_list[2] es el DataFrame generado con random: true

simulation_extended = dataframe_list[2]

#Abajo un ejemplo funcional con la segunda tanda de configuraciones
#simulation_extended = dataframe_list_example_2 [2]

# Verificar columnas y tipos de datos
print("Columnas y tipos de datos en simulation_extended:")
print(simulation_extended.dtypes)

# Convertir las columnas categóricas al tipo 'category'
for col in simulation_extended.columns:
    if simulation_extended[col].dtype == 'object':
        simulation_extended[col] = simulation_extended[col].astype('category')

# Verificar y seleccionar solo columnas categóricas
category_cols = simulation_extended.select_dtypes(include=['category']).columns.tolist()
numeric_cols = simulation_extended.select_dtypes(include=['float64', 'int64']).columns.tolist()

print("Columnas categóricas:", category_cols)
print("Columnas numéricas:", numeric_cols)

# Verificar si hay columnas categóricas disponibles
if not category_cols:
    raise ValueError("No categorical columns found in simulation_extended")

# Paso 1: Analizar combinaciones de variables categóricas y calcular probabilidades
category_combinations = simulation_extended.groupby(category_cols, observed=True).size().reset_index(name='counts')
category_combinations['prob'] = category_combinations['counts'] / category_combinations['counts'].sum()

# Paso 2: Generar el nuevo DataFrame basado en combinaciones categóricas
def get_categorical_dataset_simulated(simulation_extended, category_cols, num_records):
    category_combinations['num_samples'] = (category_combinations['prob'] * num_records).round().astype(int)
    simulated = pd.DataFrame()
    
    for _, row in category_combinations.iterrows():
        sample = pd.DataFrame([row[category_cols]] * row['num_samples'], columns=category_cols)
        simulated = pd.concat([simulated, sample], ignore_index=True)
        
    return simulated

# Generar el dataset aleatorio con 100,000 registros basado en combinaciones categóricas
num_records = 100000
final_simulation = get_categorical_dataset_simulated(simulation_extended, category_cols, num_records)

# Paso 3: Extender el DataFrame con las columnas numéricas
def get_numeric_column_simulated(simulated, df_origin, categories, column_name):
    a1 = df_origin.groupby(categories, observed=True, as_index=False).agg({
        column_name: ["min", "max", "mean", "std"]
    })
    
    nc = categories + ["Min", "Max", "Mean", "Std"]
    a1.columns = nc

    column_simulated = pd.DataFrame()
    
    for i in a1.index:
        rs = a1.loc[i]
        segment = simulated[(simulated[categories] == rs[categories].values).all(axis=1)].copy()
        # Validar parámetros de truncnorm
        mean, std, min_val, max_val = rs["Mean"], rs["Std"], rs["Min"], rs["Max"]
        if std > 0 and min_val < max_val:
            data = generate_truncated_normal_data(mean, std, min_val, max_val, segment.shape[0])
        else:
            data = np.full(segment.shape[0], mean)  # Rellenar con el valor medio si std no es válido
        segment[column_name] = data
        column_simulated = pd.concat([column_simulated, segment])

    return column_simulated.reset_index(drop=True)

# Extender el DataFrame con las columnas numéricas
for nc in numeric_cols:
    df_numeric = get_numeric_column_simulated(final_simulation, simulation_extended, category_cols, nc)
    final_simulation[nc] = df_numeric[nc]

print(final_simulation)


IndexError: list index out of range

In [26]:
# Definición de nuevas configuraciones de ejemplo
d6 = {
    "ds": "dataset6",
    "columns": [
        {
            "name": "region",
            "type": "category",
            "values": ["North", "South", "East", "West"]
        },
        {
            "name": "employee_id",
            "type": "unique"
        },
        {
            "name": "salary",
            "type": "numeric",
            "values": {"min": 30000, "max": 70000}
        },
        {
            "name": "bonus",
            "type": "numeric",
            "values": {"min": 2000, "max": 10000, "std": 1500, "mean": 5000}
        }
    ],
    "random": True,
    "random_rows": 1500
}

d7 = {
    "ds": "dataset7",
    "columns": [
        {
            "name": "employee_id",
            "type": "unique"
        },
        {
            "name": "region",
            "type": "foreign",
            "values": "dataset6.region"
        },
        {
            "name": "department",
            "type": "category",
            "values": ["HR", "Finance", "IT", "Sales"]
        },
        {
            "name": "performance_score",
            "type": "numeric",
            "values": {"min": 0, "max": 100}
        }
    ],
    "random": True,
    "random_rows": 1000
}

d8 = {
    "ds": "dataset8",
    "columns": [
        {
            "name": "employee_id",
            "type": "unique"
        },
        {
            "name": "department",
            "type": "foreign",
            "values": "dataset7.department"
        },
        {
            "name": "project_count",
            "type": "numeric",
            "values": {"min": 1, "max": 20}
        }
    ],
    "random": True,
    "random_rows": 800
}

# Lista de configuraciones para el segundo ejemplo
conf_list_example_2 = [d8, d7, d6]

# Construir los dataframes para el segundo ejemplo
dataframe_list_example_2 = build_dataframes(conf_list_example_2)

# Imprimir los dataframes generados
for df in dataframe_list_example_2:
    print(df.head(), "\n")


        employee_id  project_count
0  RaiwjeMmGnDf79il      19.063119
1  Ao7YjQdYoCvzuwJD      14.766020
2  n2yc4bDNRaygt7kM       9.400420
3  9lLdZpUzB29Y9CQD       7.971182
4  YybNlLgCle7c43EN      19.551072 

        employee_id department  performance_score
0  Y3xjLzIHmp0eFMjq         IT          75.219055
1  opOnrkjenoO8YxZs      Sales          95.357117
2  StmjFTCspvhgIPMm         IT          77.898470
3  IITS8mwmkwbJQV8b         IT          43.947202
4  30MURrJrspksGLYc      Sales          29.499710 

  region       employee_id        salary        bonus
0  South  sgQGkltkfJyqazIF  61914.898466  5376.426784
1   West  5Yeu5QKB0YM3rsha  59915.219258  5502.866241
2   East  Q8OXaXILgj3MNfyT  43277.005031  5044.840949
3   East  59L2YljapgaZ736l  37032.223715  3805.790323
4   West  3NDumNNHMvLmHvDK  35193.987042  4465.078251 

